In [15]:
import numpy as np
import pandas as pd

In [16]:
X = np.genfromtxt("hw01_data_points.csv", delimiter = ",", dtype = str)
y = np.genfromtxt("hw01_class_labels.csv", delimiter = ",", dtype = int)


In [17]:
def train_test_split(X, y):
    # your implementation starts below
    
    X_train = X[0:50000]
    X_test = X[50000:93926]
    y_train = y[0:50000]
    y_test = y[50000:93926]
    
    # your implementation ends above
    return(X_train, y_train, X_test, y_test)

In [18]:
X_train, y_train, X_test, y_test = train_test_split(X, y)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(50000, 7)
(50000,)
(43925, 7)
(43925,)


In [19]:
def estimate_prior_probabilities(y):
    # your implementation starts below
    
    K = np.max(y)
    class_priors = [np.mean(y == (c + 1)) for c in range(K)]
    
    # your implementation ends above
    return(class_priors)

In [20]:
class_priors = estimate_prior_probabilities(y_train)
print(class_priors)

[0.0452, 0.9548]


In [21]:
def estimate_nucleotide_probabilities(X, y):
    # your implementation starts below
    N = X.shape[1]
    K = np.max(y)
        
    pAcd = [np.mean(X[y == (c + 1)] == 'A',axis=0) for c in range(K)]
    pCcd = [np.mean(X[y == (c + 1)] == 'C',axis=0) for c in range(K)]
    pGcd = [np.mean(X[y == (c + 1)] == 'G',axis=0) for c in range(K)]
    pTcd = [np.mean(X[y == (c + 1)] == 'T',axis=0) for c in range(K)]
    
    pAcd = np.array(pAcd)
    pCcd = np.array(pCcd)
    pGcd = np.array(pGcd)
    pTcd = np.array(pTcd)
    
    
    # your implementation ends above
    return(pAcd, pCcd, pGcd, pTcd)

In [22]:
pAcd, pCcd, pGcd, pTcd = estimate_nucleotide_probabilities(X_train, y_train)
print(pAcd)
print(pCcd)
print(pGcd)
print(pTcd)

[[0.32345133 0.64424779 0.09424779 0.56681416 0.68938053 0.0800885
  0.16371681]
 [0.25450356 0.2728739  0.30117302 0.19522413 0.25473398 0.23990364
  0.26292417]]
[[0.37433628 0.12123894 0.02964602 0.03230088 0.08053097 0.05221239
  0.16725664]
 [0.21979472 0.2340176  0.06514453 0.20605362 0.21214914 0.24767491
  0.21723921]]
[[0.18716814 0.1199115  0.81814159 0.37477876 0.11371681 0.79513274
  0.19690265]
 [0.24805195 0.24434437 0.2676791  0.33636364 0.22869711 0.23104315
  0.23016339]]
[[0.11504425 0.11460177 0.0579646  0.02610619 0.11637168 0.07256637
  0.47212389]
 [0.27764977 0.24876414 0.36600335 0.26235861 0.30441977 0.2813783
  0.28967323]]


In [23]:
def calculate_score_values(X, pAcd, pCcd, pGcd, pTcd, class_priors):
    # your implementation starts below
    
    score = 1
    score_values = []
    
    for i in range(X.shape[0]):
        for c in range(2):
            for d in range(X.shape[1]):
                if X[i,d] == 'A':
                    score *= pAcd[c,d]
                elif X[i,d] == 'C':
                    score *= pCcd[c,d]
                elif X[i,d] == 'G':
                    score *= pGcd[c,d]
                elif X[i,d] == 'T':
                    score *= pTcd[c,d]
            score = np.log(score) + np.log(class_priors[c])
            score_values.append(score)
            score = 1
            
    score_values = np.array(score_values).reshape(X.shape[0], 2)
    

    
    # your implementation ends above
    return(score_values)


In [24]:
scores_train = calculate_score_values(X_train, pAcd, pCcd, pGcd, pTcd, class_priors)
print(scores_train)

scores_test = calculate_score_values(X_test, pAcd, pCcd, pGcd, pTcd, class_priors)
print(scores_test)


[[-15.84302871  -9.3433522 ]
 [-16.73815388  -8.90270491]
 [-17.12201508  -8.87059123]
 ...
 [-12.64747986  -9.3077559 ]
 [-18.14584244  -9.54820083]
 [-14.67706629  -9.45002167]]
[[-17.02244643  -9.95385128]
 [-13.10176993  -9.42908242]
 [-12.61507938  -9.54180473]
 ...
 [-12.86990369  -9.21875465]
 [-12.90250138  -9.38540918]
 [-16.43266951  -9.37467256]]


In [25]:
def calculate_confusion_matrix(y_truth, scores):
    # your implementation starts below
    
    y_pred = np.zeros(y_truth.shape[0])
    
    for i in range(y_truth.shape[0]):
        y_pred[i] = np.argmax(scores[i]) + 1
    confusion_matrix = pd.crosstab(y_pred, y_truth, rownames = ['y_pred'], colnames = ['y_truth'])
    
    # your implementation ends above
    return(confusion_matrix)

In [26]:
confusion_train = calculate_confusion_matrix(y_train, scores_train)
print(confusion_train)

confusion_test = calculate_confusion_matrix(y_test, scores_test)
print(confusion_test)

y_truth     1      2
y_pred              
1.0      1066    484
2.0      1194  47256
y_truth     1      2
y_pred              
1.0       891    416
2.0      1057  41561
